In [1]:
# Global imports
import hatchet as ht
import pandas as pd
import numpy as np
from scipy import stats
import utils
import random
import os
import json

import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join

from ast import literal_eval as make_list
import networkx as nx
from networkx.readwrite import json_graph

In [2]:
# Local imports
from state import State
from preprocess import PreProcess
from actions.similarity import Similarity
from actions.union_graph import UnionGraph
from pipeline import Pipeline
from hatchet_to_networkx import HatchetToNetworkX

In [3]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format

# Set Path

In [4]:
experiment = 'bcast'
dataset = 'osu_scatter.128.8.2019-10-09_04-04-22'
path = '/mnt/2TB/p/lustre1/rahim/data/hpcdbs/osu/catalyst/mvapich2/2.3/'

In [5]:
experiment = 'osu_bcast_18'
dataset = 'osu_bcast.1.18.2019-09-03_11-33-15'
path = '/home/vidi/Work/llnl/CallFlow/'

In [6]:
configPath = path + 'config_files'
dataPath = path + experiment
dotPath = path + 'data/processed'
experimentPath = dataPath + '/' + experiment
experimentDotPath = dotPath + '/' + experiment

# Read Config file

In [7]:
from configFileReader import *
# configFileName = '/home/vidi/Work/llnl/CallFlow/config_files/scatter vs scatterv.json'
configFileName = '/home/vidi/Work/llnl/CallFlow/config_files/osu_bcast-rank=10.json'
config = configFileReader(configFileName)

In [8]:
from actions.create import Create
hpctoolkit_state = Create(config, dataset)

 #   [callfow.py] Action: Creating graphframes:  "osu_bcast.1.18.2019-09-03_11-33-15"


<configFileReader: {json = {'runName': 'osu_bcast_10', 'filter_perc': 5, 'callflow_path': '/home/vidi/Work/llnl/CallFlow/', 'save_path': 'data/processed', 'datasets': [{'name': 'osu_bcast.1.10.2019-09-04_07-05-12', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-04_07-05-12', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-03_23-33-03', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-03_23-33-03', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-04_08-07-45', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-04_08-07-45', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-03_13-07-55', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-03_13-07-55', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-04_03-18-07', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-04_03-18-07', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.

KeyError: 'osu_bcast.1.18.2019-09-03_11-33-15'

In [ ]:
hpctoolkit_state.df.name.unique()

In [9]:
def read_gf(dotdir, name):
    state = State(name)
    df_filepath = dotdir + '/' + name +  '/filter_df.csv'
    entire_df_filepath = dotdir + '/' + name + '/entire_df.csv'
    graph_filepath = dotdir + '/' + name + '/filter_graph.json'
    entire_graph_filepath = dotdir + '/' + name + '/entire_graph.json'

    group_df_filepath = dotdir + '/' + name + '/group_df.csv'

    with open(entire_graph_filepath, 'r') as entire_graphFile:
        entire_data = json.load(entire_graphFile)
        
    state.gf = ht.GraphFrame.from_literal(entire_data)
    state.df = pd.read_csv(entire_df_filepath)
    state.graph = state.gf.graph

    return state

# Create the props file

In [10]:
def dump_props_file(directory):
    states = {}
    files = listdir(directory)
    ret = []
    for file in files:
        if file != 'ensemble_df.csv' and file != 'ensemble_graph.json' and file != 'similarity.json':
#             state = read_gf(directory, file)
#             nop = len(state.df['rank'].unique())
#             states[file + ':' + str(nop)] = state
            if(True):
                ret.append({
                    "name": file,
                    "path": "./data/osu_bcast_10/"+ file,
                    "format": "hpctoolkit",
                    "props": {
                    },
                    "nop": 1024
                })
    print(json.dumps({
        "datasets": ret }))
print(experimentDotPath)
dump_props_file(dataPath)

/home/vidi/Work/llnl/CallFlow/data/processed/osu_bcast_18


FileNotFoundError: [Errno 2] No such file or directory: '/home/vidi/Work/llnl/CallFlow/osu_bcast_18'

# Read all states from a directory

In [11]:
def read_gfs(directory):
    states = {}
    files = listdir(directory)
    for file in files:
        if file != 'ensemble_df.csv' and file != 'ensemble_graph.json' and file != 'similarity.json':
            state = read_gf(directory, file)
            states[file] = state
    return states

In [12]:
states = read_gfs(experimentDotPath)

# Convert hatchet graph to networkX graph

In [14]:
dataset1 = 'osu_bcast.1.18.2019-09-03_11-33-15'
dataset2 = 'osu_bcast.1.18.2019-09-03_12-29-49'
dataset3 = 'osu_bcast.1.18.2019-09-03_10-33-44'

In [ ]:
networkX_graphs = {}
for idx, dataset in enumerate(states):
    state = states[dataset]
    networkX_graphs[dataset] = HatchetToNetworkX(state, 'path', construct_graph=True, add_data=False)
    print(networkX_graphs[dataset].g.nodes())

# Union of all graphs.

In [ ]:
u_graph = UnionGraph()
u_df = pd.DataFrame()
for idx, dataset in enumerate(networkX_graphs):
    u_graph.unionize(networkX_graphs[dataset].g, dataset)
    u_df = pd.concat([u_df, networkX_graphs[dataset].df])

# Compare and check how it looks different with other graphs.

In [ ]:
def printEdges(nxGraph):
    print("Total number of edges: ", len(nxGraph.edges()))
    for edge in nxGraph.edges():
        print(edge[0], '=> ', edge[1])

In [ ]:
printEdges(networkX_graphs[dataset1].g)

In [ ]:
printEdges(networkX_graphs[dataset2].g)

In [ ]:
printEdges(u_graph.R)

In [ ]:
print(u_df.info(), networkX_graphs[dataset].df.info())

In [ ]:
print(u_df.groupby(['dataset', 'name']).count())

In [ ]:
def print_df_count(df):
    print(df.groupby(['name']).count())

In [ ]:
for state in networkX_graphs:
    print_df_count(networkX_graphs[state].df)

# Filter the union graph and also be distribution aware.

In [ ]:
class FilterUnion():
    def __init__(self, state):
        self.df = state.df
        self.dataset_df = self.df.groupby(['dataset'])
        self.dataset_idx = {}
        self.set_max_min_times()
    
    def set_max_min_times(self):
        self.max_time_inc_list = np.array([])
        self.min_time_inc_list = np.array([])
        self.max_time_exc_list = np.array([])
        self.min_time_exc_list = np.array([])
        count = 0
        for dataset, df in self.dataset_df:
            self.dataset_idx[dataset] = count
            self.max_time_inc_list = np.hstack([self.max_time_inc_list, df['time (inc)'].max()])
            self.min_time_inc_list = np.hstack([self.min_time_inc_list, df['time (inc)'].min()])
            self.max_time_exc_list = np.hstack([self.max_time_exc_list, df['time'].max()])
            self.min_time_exc_list = np.hstack([self.min_time_exc_list, df['time'].min()])
            count += 1
        print("Dataset idx: ", self.dataset_idx)
        print("Min. time (inc): ", self.min_time_inc_list)
        print("Max. time (inc): ", self.max_time_inc_list)
        print("Min. time (exc): ", self.min_time_exc_list)
        print("Max. time (exc): ", self.max_time_exc_list)
        self.max_time_inc = np.max(self.max_time_inc_list)
        self.min_time_inc = np.min(self.min_time_inc_list)
        self.max_time_exc = np.max(self.max_time_exc_list)
        self.min_time_exc = np.min(self.min_time_exc_list)
        
    def filter_time_inc_overall(self, perc):
        df = self.df.loc[self.df['time (inc)'] > perc*0.01*self.max_time_inc]
        filter_call_sites = df['name'].unique()
        print("Number of nodes left in dataframe: ", len(filter_call_sites))
        return df[df['name'].isin(filter_call_sites)]
    
    def filter_graph_nodes(self, df, g):
        call_sites = df['name'].unique()
    
        ret = nx.DiGraph()
        
        for edge in g.edges():
            if edge[0] in call_sites and edge[1] in call_sites:
                ret.add_edge(edge[0], edge[1])
                
        return ret
    
    def generic_map(self, df, g):
        for column in df.columns:
            values = {}
            for node in g.nodes():
                values[node] = df.loc[df['name'] == node][column].tolist()
            nx.set_node_attributes(g, name=column, values = values)
    
    def add_node_attributes(self, df, g):            
        generic_map = self.generic_map(df, g)
        print(g.nodes(data=True))

In [ ]:
states = {}
states['ensemble'] = State('ensemble')
states['ensemble'].df = u_df
states['ensemble'].g = u_graph.R

union_filter = FilterUnion(states['ensemble'])

In [ ]:
states['ensemble'].df = union_filter.filter_time_inc_overall(5)

In [ ]:
states['ensemble'].g = union_filter.filter_graph_nodes(states['ensemble'].df, states['ensemble'].g)

In [ ]:
print(len(states['ensemble'].g.nodes()), len(states['ensemble'].g.edges()))

In [ ]:
union_filter.add_node_attributes(states['ensemble'].df, states['ensemble'].g)

# Check if the grouped dataframe is right.

In [ ]:
check_group_df = pd.read_csv('/home/vidi/Work/llnl/CallFlow/data/processed/osu_bcast_18/ensemble_df.csv')

In [ ]:
print(check_group_df['module'].unique())

# Pipeline

In [17]:
from pipeline import Pipeline
datasets = [dataset1, dataset2, dataset3]
pipeline = Pipeline(config)

In [18]:
for idx, dataset_name in enumerate(datasets):
    states[dataset_name] = pipeline.create(dataset_name)
    states[dataset_name] = pipeline.process(states[dataset_name], 'entire')
    states[dataset_name] = pipeline.convertToNetworkX(states[dataset_name], 'path')
    pipeline.write_gf(states[dataset_name], dataset_name, 'entire')
    states[dataset_name] = pipeline.filterNetworkX(states, dataset_name, config.filter_perc)
    pipeline.write_dataset_gf(states[dataset_name], dataset_name, 'filter')

 #   [callfow.py] Action: Creating graphframes:  "osu_bcast.1.18.2019-09-03_11-33-15"


<configFileReader: {json = {'runName': 'osu_bcast_10', 'filter_perc': 5, 'callflow_path': '/home/vidi/Work/llnl/CallFlow/', 'save_path': 'data/processed', 'datasets': [{'name': 'osu_bcast.1.10.2019-09-04_07-05-12', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-04_07-05-12', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-03_23-33-03', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-03_23-33-03', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-04_08-07-45', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-04_08-07-45', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-03_13-07-55', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-03_13-07-55', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.10.2019-09-04_03-18-07', 'path': './data/osu_bcast_10/osu_bcast.1.10.2019-09-04_03-18-07', 'format': 'hpctoolkit', 'props': {}, 'nop': 10}, {'name': 'osu_bcast.1.

KeyError: 'osu_bcast.1.18.2019-09-03_11-33-15'

In [ ]:
states['ensemble'] = pipeline.union(states)
states['ensemble'] = pipeline.filterNetworkX(states, 'ensemble', config.filter_perc)

# Group the filtered union graph based on module. 

In [ ]:
# Pipeline for the ensemble graph
states['ensemble'] = pipeline.group(states, dataset_name, 'module')
pipeline.write_ensemble_gf(states, 'ensemble')